# Datasets - MIMIC-IV and eICU-CRD

## Feature Extract

In [6]:
%cd /content/drive/MyDrive/TAU/Datasets

/content/drive/MyDrive/TAU/Datasets


In [ ]:
!pip install gsutil

In [5]:
!gsutil cp -r gs://mimic_extract/all_hourly_data.h5 /content/drive/MyDrive/TAU/Datasets/all_hourly_data.h5

ServiceException: 401 Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object.


In [12]:
!wget -r -N -c -np --user pavanreddy --ask-password https://storage.cloud.google.com/mimic_extract/all_hourly_data.h5

Password for user ‘pavanreddy’: 
--2022-06-08 10:49:52--  https://storage.cloud.google.com/mimic_extract/all_hourly_data.h5
Resolving storage.cloud.google.com (storage.cloud.google.com)... 108.177.127.102, 108.177.127.100, 108.177.127.138, ...
Connecting to storage.cloud.google.com (storage.cloud.google.com)|108.177.127.102|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://accounts.google.com/ServiceLogin?service=cds&passive=1209600&continue=https://storage.cloud.google.com/mimic_extract/all_hourly_data.h5&followup=https://storage.cloud.google.com/mimic_extract/all_hourly_data.h5 [following]
--2022-06-08 10:49:52--  https://accounts.google.com/ServiceLogin?service=cds&passive=1209600&continue=https://storage.cloud.google.com/mimic_extract/all_hourly_data.h5&followup=https://storage.cloud.google.com/mimic_extract/all_hourly_data.h5
Resolving accounts.google.com (accounts.google.com)... 173.194.69.84, 2a00:1450:4013:c00::54
Connecting to accounts.goog

In [13]:
import h5py

In [15]:
filename = "/content/drive/MyDrive/TAU/Datasets/all_hourly_data.h5"
feature_extract = h5py.File(filename, 'r')

OSError: ignored

## Packages

In [1]:
import os
import pandas as pd
import numpy as np

## Downloading and Extracting Data

### Creating Directories

In [2]:
%cd drive
%pwd

/content/drive


'/content/drive'

In [3]:
%cd MyDrive/TAU

/content/drive/MyDrive/TAU


In [ ]:
%mkdir Datasets

### Extracting eICU Dataset - To Drive

In [ ]:
# !wget -r -N -c -np --user pavanreddy --ask-password https://physionet.org/files/eicu-crd/2.0/

### Extracting MIMIC-IV Dataset - To Drive

In [ ]:
%cd Datasets

/content/drive/MyDrive/TAU/Datasets


In [ ]:
# !wget -r -N -c -np --user pavanreddy --ask-password https://physionet.org/files/mimiciv/1.0/

### Extracting MIMIC-III

In [ ]:
# !wget -r -N -c -np --user pavanreddy --ask-password https://physionet.org/files/mimiciii/1.4/

In [ ]:
!git clone https://github.com/MIT-LCP/mimic-code.git

Cloning into 'mimic-code'...
remote: Enumerating objects: 8888, done.
remote: Counting objects: 100% (864/864), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 8888 (delta 751), reused 687 (delta 679), pack-reused 8024
Receiving objects: 100% (8888/8888), 19.14 MiB | 6.22 MiB/s, done.
Resolving deltas: 100% (5671/5671), done.
Checking out files: 100% (522/522), done.


In [ ]:
%cd mimic-code/mimic-iii/buildmimic/postgres

/content/drive/MyDrive/TAU/mimic-code/mimic-iii/buildmimic/postgres


In [ ]:
!make help

In [ ]:
! sudo apt install postgresql postgresql-contrib
!make create-user mimic datadir="/content/drive/MyDrive/TAU/Datasets/mimiciii/1.4"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
postgresql is already the newest version (10+190ubuntu0.1).
postgresql-contrib is already the newest version (10+190ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
psql: could not connect to server: No such file or directory
	Is the server running locally and accepting
	connections on Unix domain socket "/var/run/postgresql/.s.PGSQL.5432"?
/usr/bin/test: missing argument after ‘100000’
-------------------------------------
-- Creating user, database, schema --
-------------------------------------

MIMIC_USER="postgres" MIMIC_DB="mimic" MIMIC_PASSWORD="postgres" MIMIC_SCHEMA="mimiciii" DBHOST="" DBPORT="" ./create_mimic_user.sh
/bin/sh: 1: ./create_mimic_user.sh: Permission denied
Makefile:76: recipe for target 'create-user' f

In [ ]:
! sudo -u postgres createuser mimic

could not change directory to "/content/drive/MyDrive/TAU/mimic-code/mimic-iii/buildmimic/postgres": Permission denied
createuser: could not connect to database template1: could not connect to server: No such file or directory
	Is the server running locally and accepting
	connections on Unix domain socket "/var/run/postgresql/.s.PGSQL.5432"?


## Data Access

### Big Query Access

In [ ]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import tensorflow as tf

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

# Imports for accessing Datathon data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
auth.authenticate_user()

In [ ]:
client = bigquery.Client(project='physionet-data')
datasets = client.list_datasets()
for dataset in datasets:
  did = dataset.dataset_id
  print('Dataset "%s" has the following tables:' % did)
#   for table in client.list_tables(client.dataset(did)):
#     print('    ' + table.table_id)

Dataset "eicu_crd" has the following tables:
Dataset "eicu_crd_demo" has the following tables:
Dataset "eicu_crd_derived" has the following tables:
Dataset "mimic_core" has the following tables:
Dataset "mimic_derived" has the following tables:
Dataset "mimic_hosp" has the following tables:
Dataset "mimic_icu" has the following tables:
Dataset "mimiciii_demo" has the following tables:


In [ ]:
job = client.query("""
    SELECT
    *
    FROM
    `physionet-data.mimic_core.admissions`
    LIMIT 10
""")

results = job.result()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



Forbidden: ignored

In [ ]:
def run_query(query):
  return pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, configuration={'query':{'useLegacySql': False}})

In [ ]:
project_id='datathon-client-00' # @param
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

In [ ]:
def run_query(query):
  return pd.io.gbq.read_gbq(query, project_id=project_id, configuration={'query':{'useLegacySql': False}})

In [ ]:
run_query('''
SELECT
  *
FROM
  `physionet-data.mimic_core.admissions`
LIMIT 10
''')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



GenericGBQException: ignored

### Google Drive

#### Setting Up

In [ ]:
DATA_DIR = '/content/drive/MyDrive/TAU/Datasets'
eICU_DIR = DATA_DIR + '/eicu-crd'
mimiciv_DIR = DATA_DIR + '/mimiciv'

In [ ]:
for (r,d,f) in os.walk(DATA_DIR):
    print(r)
    print(d)
    print(f)
    print()

/content/drive/MyDrive/TAU/Datasets
['eicu-crd', 'mimiciv']
[]

/content/drive/MyDrive/TAU/Datasets/eicu-crd
[]
['index.html', 'LICENSE.txt', 'SHA256SUMS.txt', 'admissionDrug.csv.gz', 'admissionDx.csv.gz', 'allergy.csv.gz', 'apacheApsVar.csv.gz', 'apachePatientResult.csv.gz', 'apachePredVar.csv.gz', 'carePlanCareProvider.csv.gz', 'carePlanEOL.csv.gz', 'carePlanGeneral.csv.gz', 'carePlanGoal.csv.gz', 'carePlanInfectiousDisease.csv.gz', 'customLab.csv.gz', 'diagnosis.csv.gz', 'hospital.csv.gz', 'infusionDrug.csv.gz', 'intakeOutput.csv.gz', 'lab.csv.gz', 'medication.csv.gz', 'microLab.csv.gz', 'note.csv.gz', 'nurseAssessment.csv.gz', 'nurseCare.csv.gz', 'nurseCharting.csv.gz', 'pastHistory.csv.gz', 'patient.csv.gz', 'physicalExam.csv.gz', 'respiratoryCare.csv.gz', 'respiratoryCharting.csv.gz', 'treatment.csv.gz', 'vitalAperiodic.csv.gz', 'vitalPeriodic.csv.gz']

/content/drive/MyDrive/TAU/Datasets/mimiciv
['core', 'hosp', 'icu']
['index.html', 'LICENSE.txt', 'SHA256SUMS.txt']

/content/dr

#### MIMIC-IV - CORE

In [ ]:
admissions = pd.read_csv(os.path.join(mimiciv_DIR, 'core', 'admissions.csv.gz'))
admissions.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,14679932,21038362,2139-09-26 14:16:00,2139-09-28 11:30:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,SINGLE,UNKNOWN,NaN,NaN,0
1,15585972,24941086,2123-10-07 23:56:00,2123-10-12 11:22:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,WHITE,NaN,NaN,0
2,11989120,21965160,2147-01-14 09:00:00,2147-01-17 14:25:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,UNKNOWN,NaN,NaN,0
3,17817079,24709883,2165-12-27 17:33:00,2165-12-31 21:18:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,OTHER,NaN,NaN,0
4,15078341,23272159,2122-08-28 08:48:00,2122-08-30 12:32:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,0


In [ ]:
patients = pd.read_csv(os.path.join(mimiciv_DIR, 'core', 'patients.csv.gz'))
patients.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000048,F,23,2126,2008 - 2010,NaN
1,10002723,F,0,2128,2017 - 2019,NaN
2,10003939,M,0,2184,2008 - 2010,NaN
3,10004222,M,0,2161,2014 - 2016,NaN
4,10005325,F,0,2154,2011 - 2013,NaN


#### MIMIC-IV - HOSP

In [ ]:
diagnoses_icd = pd.read_csv(os.path.join(mimiciv_DIR, 'hosp', 'diagnoses_icd.csv.gz'))
diagnoses_icd.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,15734973,20475282,3,2825,9
1,15734973,20475282,2,V0251,9
2,15734973,20475282,5,V270,9
3,15734973,20475282,1,64891,9
4,15734973,20475282,4,66481,9


#### MIMIC-IV - ICU

In [ ]:
icustays = pd.read_csv(os.path.join(mimiciv_DIR, 'icu', 'icustays.csv.gz'))
icustays.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,17867402,24528534,31793211,Trauma SICU (TSICU),Trauma SICU (TSICU),2154-03-03 04:11:00,2154-03-04 18:16:56,1.587454
1,14435996,28960964,31983544,Trauma SICU (TSICU),Trauma SICU (TSICU),2150-06-19 17:57:00,2150-06-22 18:33:54,3.025625
2,17609946,27385897,33183475,Trauma SICU (TSICU),Trauma SICU (TSICU),2138-02-05 18:54:00,2138-02-15 12:42:05,9.741725
3,18966770,23483021,34131444,Trauma SICU (TSICU),Trauma SICU (TSICU),2123-10-25 10:35:00,2123-10-25 18:59:47,0.350544
4,12776735,20817525,34547665,Neuro Stepdown,Neuro Stepdown,2200-07-12 00:33:00,2200-07-13 16:44:40,1.674769
